In [5]:
!pip install qiskit
!pip install qiskit_aer

In [6]:
import matplotlib.pyplot as plt
import numpy as np
from qiskit import QuantumCircuit, transpile, assemble
from qiskit_aer import AerSimulator
from math import gcd
from numpy.random import randint
import pandas as pd
from fractions import Fraction
print("Imports Successful")

Imports Successful


In [7]:
import numpy as np
from qiskit import QuantumCircuit

def create_phase_rotation_gate(n):
    """
    Create a controlled phase rotation gate for a given level n.

    Parameters:
        n (int): Determines the angle of the phase rotation.

    Returns:
        Gate: A controlled phase rotation gate.
    """
    # Calculate the rotation angle
    theta = 2 * np.pi / (2 ** n)

    # Define the single-qubit phase rotation gate
    phase_gate = QuantumCircuit(1, name=f'R_{n}')
    phase_gate.rz(theta, 0)

    # Convert to a gate and make it controlled
    controlled_phase_gate = phase_gate.to_gate().control(1)
    return controlled_phase_gate



Implementing QFT

In [8]:
def quantum_fourier_transform(num_qubits):
    """
    Create a Quantum Fourier Transform (QFT) circuit for a given number of qubits.

    Parameters:
        num_qubits (int): Number of qubits in the QFT circuit.

    Returns:
        QuantumCircuit: The QFT circuit.
    """
    # Initialize the QFT circuit
    qft_circuit = QuantumCircuit(num_qubits, name="QFT")

    # Apply the QFT algorithm
    for target_qubit in range(num_qubits):
        # Apply Hadamard gate to the target qubit
        qft_circuit.h(target_qubit)

        # Apply controlled phase rotation gates
        for control_qubit in range(target_qubit + 1, num_qubits):
            qft_circuit.append(
                create_phase_rotation_gate(control_qubit - target_qubit),
                [control_qubit, target_qubit]
            )

        # Add a barrier for clarity in the circuit diagram
        qft_circuit.barrier()

    # Swap qubits to reverse their order
    for i in range(num_qubits // 2):
        qft_circuit.swap(i, num_qubits - i - 1)

    return qft_circuit

# Example usage
num_qubits = 5
qft_circuit = quantum_fourier_transform(num_qubits)
print(qft_circuit.draw())


     ┌───┐┌─────┐┌─────┐┌─────┐┌─────┐ ░                            ░      »
q_0: ┤ H ├┤ R_1 ├┤ R_2 ├┤ R_3 ├┤ R_4 ├─░────────────────────────────░──────»
     └───┘└──┬──┘└──┬──┘└──┬──┘└──┬──┘ ░ ┌───┐┌─────┐┌─────┐┌─────┐ ░      »
q_1: ────────■──────┼──────┼──────┼────░─┤ H ├┤ R_1 ├┤ R_2 ├┤ R_3 ├─░──────»
                    │      │      │    ░ └───┘└──┬──┘└──┬──┘└──┬──┘ ░ ┌───┐»
q_2: ───────────────■──────┼──────┼────░─────────■──────┼──────┼────░─┤ H ├»
                           │      │    ░                │      │    ░ └───┘»
q_3: ──────────────────────■──────┼────░────────────────■──────┼────░──────»
                                  │    ░                       │    ░      »
q_4: ─────────────────────────────■────░───────────────────────■────░──────»
                                       ░                            ░      »
«                    ░              ░       ░       
«q_0: ───────────────░──────────────░───────░──X────
«                    ░              ░       ░  

Inverse QFT (QFT dagger)

In [9]:
import numpy as np
from qiskit import QuantumCircuit

def create_inverse_phase_rotation_gate(n):
    """
    Create a controlled inverse phase rotation gate for a given level n.

    Parameters:
        n (int): Determines the angle of the phase rotation.

    Returns:
        Gate: A controlled inverse phase rotation gate.
    """
    # Calculate the inverse rotation angle (negated)
    theta = -2 * np.pi / (2 ** n)

    # Define the single-qubit inverse phase rotation gate
    inverse_phase_gate = QuantumCircuit(1, name=f'R_{n}^\u2020')  # \u2020 for dagger (†)
    inverse_phase_gate.rz(theta, 0)

    # Convert to a gate and make it controlled
    controlled_inverse_phase_gate = inverse_phase_gate.to_gate().control(1)
    return controlled_inverse_phase_gate



In [10]:

def quantum_fourier_transform_inverse(num_qubits):
    """
    Create an Inverse Quantum Fourier Transform (QFT†) circuit for a given number of qubits.

    Parameters:
        num_qubits (int): Number of qubits in the QFT† circuit.

    Returns:
        QuantumCircuit: The QFT† circuit.
    """
    # Initialize the QFT† circuit
    qft_inverse_circuit = QuantumCircuit(num_qubits, name="QFT†")

    # Reverse the qubit swap operations
    for i in range(num_qubits // 2):
        qft_inverse_circuit.swap(i, num_qubits - i - 1)

    # Add a barrier for clarity in the circuit diagram
    qft_inverse_circuit.barrier()

    # Apply the inverse QFT algorithm
    for target_qubit in reversed(range(num_qubits)):
        # Apply controlled inverse phase rotation gates
        for control_qubit in reversed(range(target_qubit + 1, num_qubits)):
            qft_inverse_circuit.append(
                create_inverse_phase_rotation_gate(control_qubit - target_qubit),
                [control_qubit, target_qubit]
            )

        # Apply Hadamard gate to the target qubit
        qft_inverse_circuit.h(target_qubit)

        # Add a barrier for clarity in the circuit diagram
        qft_inverse_circuit.barrier()

    return qft_inverse_circuit

# Example usage
num_qubits = 4
qft_inverse_circuit = quantum_fourier_transform_inverse(num_qubits)
print(qft_inverse_circuit.draw())

            ░       ░                ░                         ░ ┌───────┐»
q_0: ─X─────░───────░────────────────░─────────────────────────░─┤ R_3^† ├»
      │     ░       ░                ░ ┌───────┐┌───────┐┌───┐ ░ └───┬───┘»
q_1: ─┼──X──░───────░────────────────░─┤ R_2^† ├┤ R_1^† ├┤ H ├─░─────┼────»
      │  │  ░       ░ ┌───────┐┌───┐ ░ └───┬───┘└───┬───┘└───┘ ░     │    »
q_2: ─┼──X──░───────░─┤ R_1^† ├┤ H ├─░─────┼────────■──────────░─────┼────»
      │     ░ ┌───┐ ░ └───┬───┘└───┘ ░     │                   ░     │    »
q_3: ─X─────░─┤ H ├─░─────■──────────░─────■───────────────────░─────■────»
            ░ └───┘ ░                ░                         ░          »
«     ┌───────┐┌───────┐┌───┐ ░ 
«q_0: ┤ R_2^† ├┤ R_1^† ├┤ H ├─░─
«     └───┬───┘└───┬───┘└───┘ ░ 
«q_1: ────┼────────■──────────░─
«         │                   ░ 
«q_2: ────■───────────────────░─
«                             ░ 
«q_3: ────────────────────────░─
«                             ░ 


 Simple Implementation for QFT_inverse

In [11]:
def qft_dagger(n):
    """n-qubit QFTdagger the first n qubits in circ"""
    qc = QuantumCircuit(n)
    # Don't forget the Swaps!
    for qubit in range(n//2):
        qc.swap(qubit, n-qubit-1)
    for j in range(n):
        for m in range(j):
            qc.cp(-np.pi/float(2**(j-m)), m, j)
        qc.h(j)
    qc.name = "QFT†"
    return qc

$$ f(x) = a^x \bmod{N}$$
where $a$ and $N$ are positive integers, $a$ is less than $N$, and they have no common factors. The period, or order ($r$), is the smallest (non-zero) integer such that:

$$a^r \bmod N = 1 $$
$$U|y\rangle = |ay\bmod N\rangle $$

To create $U^x$, we will simply repeat the circuit $x$ times.
 In the next section we will discuss a general method for creating these circuits efficiently. For all the below implementation of Shor's algo we will be using $N=15$.<br>
 The function `c_amod15` returns the controlled-U gate for `a`, repeated `power` times.

In [12]:
def c_amod15(a, power):
    """Controlled multiplication by a mod 15"""
    if a not in [2,4,7,8,11,13]:
        raise ValueError("'a' must be 2,4,7,8,11 or 13")
    U = QuantumCircuit(4)
    for iteration in range(power):
        if a in [2,13]:
            U.swap(2,3)
            U.swap(1,2)
            U.swap(0,1)
        if a in [7,8]:
            U.swap(0,1)
            U.swap(1,2)
            U.swap(2,3)
        if a in [4, 11]:
            U.swap(1,3)
            U.swap(0,2)
        if a in [7,11,13]:
            for q in range(4):
                U.x(q)
    U = U.to_gate()
    U.name = "%i^%i mod 15" % (a, power)
    c_U = U.control()
    return c_U

In [13]:
result123 = qft_dagger(4)
result123.draw(fold=-1)

┌───┐                                                                           
q_0: ─X─┤ H ├─■──────────────■───────────────────■──────────────────────────────────────
      │ └───┘ │P(-π/2) ┌───┐ │                   │                                      
q_1: ─┼───X───■────────┤ H ├─┼─────────■─────────┼──────────────■───────────────────────
      │   │            └───┘ │P(-π/4)  │P(-π/2)  │        ┌───┐ │                       
q_2: ─┼───X──────────────────■─────────■─────────┼────────┤ H ├─┼─────────■─────────────
      │                                          │P(-π/8) └───┘ │P(-π/4)  │P(-π/2) ┌───┐
q_3: ─X──────────────────────────────────────────■──────────────■─────────■────────┤ H ├
                                                                                   └───┘

shor's algo

In [14]:
N = 15
a=7 #other possible values: 2, 4, 7, 8, 11, 13

Next we quickly check it isn't already a non-trivial factor of $N$:

In [15]:
from math import gcd # greatest common divisor
gcd(a, N)

1

Simple Implementation of Quantum Phase Estimation for order finding

In [16]:
def qpe_amod15(a):
    n_count = 8
    qc = QuantumCircuit(4+n_count, n_count)
    for q in range(n_count):
        qc.h(q)     # Initialize counting qubits in state |+>
    qc.x(3+n_count) # And auxiliary register in state |1>
    for q in range(n_count): # Do controlled-U operations
        qc.append(c_amod15(a, 2**q),
                 [q] + [i+n_count for i in range(4)])
    qc.append(qft_dagger(n_count), range(n_count)) # Do inverse-QFT
    qc.measure(range(n_count), range(n_count))
    # Simulate Results
    # aer_sim = Aer.get_backend('aer_simulator')
    aer_sim = AerSimulator()
    # Setting memory=True below allows us to see a list of each sequential reading
    t_qc = transpile(qc, aer_sim)
    # qobj = assemble(t_qc, shots=1)
    result = aer_sim.run(t_qc, memory=True).result()
    readings = result.get_memory()
    print("Register Reading: " + readings[0])
    phase = int(readings[0],2)/(2**n_count)
    print("Corresponding Phase: %f" % phase)
    return phase

From this phase, we can easily find a guess for $r$:

In [17]:
phase = qpe_amod15(a) # Phase = s/r
Fraction(phase).limit_denominator(15) # Denominator should tell us r

Register Reading: 11000000
Corresponding Phase: 0.750000


Fraction(3, 4)

In [18]:
frac = Fraction(phase).limit_denominator(15)
s, r = frac.numerator, frac.denominator
print(r)

4


Now we have $r$, we might be able to use this to find a factor of $N$. Since:

$$a^r \bmod N = 1 $$

then:

$$(a^r - 1) \bmod N = 0 $$

which means $N$ must divide $a^r-1$. And if $r$ is also even, then we can write:

$$a^r -1 = (a^{r/2}-1)(a^{r/2}+1)$$

(if $r$ is not even, we cannot go further and must try again with a different value for $a$). There is then a high probability that the greatest common divisor of $N$ and either $a^{r/2}-1$, or $a^{r/2}+1$ is a proper factor of $N$ [2]:

In [19]:
guesses = [gcd(a**(r//2)-1, N), gcd(a**(r//2)+1, N)]
print(guesses)

[3, 5]


The cell below repeats the algorithm until at least one factor of 15 is found. Re-running the cell with different 'a' values(2,4,7,8,11,13) a few times shows us how the algo behaves.

In [22]:
a = 13
factor_found = False
attempt = 0
while not factor_found:
    attempt += 1
    print("\nAttempt %i:" % attempt)
    phase = qpe_amod15(a) # Phase = s/r
    frac = Fraction(phase).limit_denominator(N) # Denominator should (hopefully!) tell us r
    r = frac.denominator
    print("Result: r = %i" % r)
    if phase != 0:
        # Guesses for factors are gcd(x^{r/2} ±1 , 15)
        guesses = [gcd(a**(r//2)-1, N), gcd(a**(r//2)+1, N)]
        print("Guessed Factors: %i and %i" % (guesses[0], guesses[1]))
        for guess in guesses:
            if guess not in [1,N] and (N % guess) == 0: # Check to see if guess is a factor
                print("*** Non-trivial factor found: %i ***" % guess)
                factor_found = True


Attempt 1:
Register Reading: 01000000
Corresponding Phase: 0.250000
Result: r = 4
Guessed Factors: 3 and 5
*** Non-trivial factor found: 3 ***
*** Non-trivial factor found: 5 ***
